We need to set up some global attributes.

In [ ]:
import sagemaker
from sagemaker import get_execution_role

role = get_execution_role()
print('Role: ' + role)
sess = sagemaker.Session()

from sagemaker.amazon.amazon_estimator import get_image_uri

training_image = get_image_uri(sess.boto_region_name, 'object-detection', repo_version="latest")
print ('Training image: ' + training_image)

validation_size = 2000
print ('Validation size: ' + str(validation_size))

input_bucket_name = 'mcwhirter-airbus-ship-detection-data'
output_bucket_name = ''

train_channel = 'training'
validation_channel = 'validation'
train_annotation_channel = 'trainimages_annotation'
validation_annotation_channel = 'validationimages_annotation'

If you're happy with the validation size, let's move a subset of training data for validation purposes.

In [ ]:
%%time
import boto3

s3 = boto3.resource('s3')
bucket = s3.Bucket(input_bucket_name)

validation_jsons = bucket.objects.filter(Prefix=validation_annotation_channel)
validation_count = sum(1 for _ in validation_jsons)

move_limit = max(0, validation_size-validation_count)
training_jsons = bucket.objects.filter(Prefix=train_annotation_channel).limit(move_limit)
training_count = sum(1 for _ in training_jsons)

print ('Found {} validation images with annotation files and will move another {} from training'.format(validation_count,move_limit))

def copy_object(input_bucket_name,to_name,from_name):
    try:
        response = s3.Object(input_bucket_name, to_name).copy_from(CopySource=input_bucket_name+'/'+from_name)
#         print('Copied object {} to {}'.format(from_name,to_name))
        return response
    except s3.meta.client.exceptions.NoSuchKey:
        print('No such key')

# Move a subset of training data for validation.
if move_limit > 0:
    for filename in training_jsons:
        file = filename.key.strip(train_annotation_channel).strip('.json')
        train_image = '{}/{}.jpg'.format(train_channel,file)
        validation_image = '{}/{}.jpg'.format(validation_channel,file)
        copy_object(input_bucket_name,validation_image,train_image)

        train_json = '{}/{}.json'.format(train_annotation_channel,file)
        validation_json = '{}/{}.json'.format(validation_annotation_channel,file)
        copy_object(input_bucket_name,validation_json,train_json)

        s3.Object(input_bucket_name, train_image).delete()
        s3.Object(input_bucket_name, train_json).delete()
#         print('Deleted training data. Move complete.')

Set up variables to pass S3 paths to SageMaker

In [ ]:
s3_train_data = 's3://{}/{}'.format(input_bucket_name, train_channel)
s3_validation_data = 's3://{}/{}'.format(input_bucket_name, validation_channel)
s3_train_annotation = 's3://{}/{}'.format(input_bucket_name, train_annotation_channel)
s3_validation_annotation = 's3://{}/{}'.format(input_bucket_name, validation_annotation_channel)

print(s3_train_data)
print(s3_validation_data)
print(s3_train_annotation)
print(s3_validation_annotation)

In [ ]:
s3_output_location = 's3://{}/output'.format(output_bucket_name)

Create the SageMaker instance

In [ ]:
od_model = sagemaker.estimator.Estimator(training_image,
                                         role, 
                                         train_instance_count=1, 
                                         train_instance_type='ml.p3.2xlarge',
                                         train_volume_size = 50,
                                         train_max_run = 360000,
                                         input_mode = 'File',
                                         output_path=s3_output_location,
                                         sagemaker_session=sess)

Configure hyperparameters (that you'll likely want to optimize later)

In [ ]:
od_model.set_hyperparameters(base_network='resnet-50',
                             use_pretrained_model=1,
                             num_classes=2,
                             mini_batch_size=16,
                             epochs=5,
                             learning_rate=0.001,
                             lr_scheduler_step='10',
                             lr_scheduler_factor=0.1,
                             optimizer='sgd',
                             momentum=0.9,
                             weight_decay=0.0005,
                             overlap_threshold=0.5,
                             nms_threshold=0.45,
                             image_shape=512,
                             label_width=350,
                             num_training_samples=2000)

Tell SageMaker about the data channels we specified earlier

In [ ]:
train_data = sagemaker.session.s3_input(s3_train_data, distribution='FullyReplicated', 
                        content_type='image/jpeg', s3_data_type='S3Prefix')
validation_data = sagemaker.session.s3_input(s3_validation_data, distribution='FullyReplicated', 
                        content_type='image/jpeg', s3_data_type='S3Prefix')
train_annotation = sagemaker.session.s3_input(s3_train_annotation, distribution='FullyReplicated', 
                        content_type='image/jpeg', s3_data_type='S3Prefix')
validation_annotation = sagemaker.session.s3_input(s3_validation_annotation, distribution='FullyReplicated', 
                        content_type='image/jpeg', s3_data_type='S3Prefix')

data_channels = {'train': train_data, 'validation': validation_data, 
                 'train_annotation': train_annotation, 'validation_annotation':validation_annotation}

## Things get real...🤑 ##

Here we are going to run the training job with everything we set up previously.

Takes about 41 minutes with current config.

In [ ]:
%%time
od_model.fit(inputs=data_channels, logs=True)

Let's put the model you just trained into the wild!

In [ ]:
object_detector = od_model.deploy(initial_instance_count = 1,
                                 instance_type = 'ml.m4.xlarge')

Time to test our work. Grab an image...

In [ ]:
!wget -O test.jpg 'https://storage.googleapis.com/kaggle-media/competitions/Airbus/ships.jpg'
file_name = 'test.jpg'

with open(file_name, 'rb') as image:
    f = image.read()
    b = bytearray(f)
    ne = open('n.txt','wb')
    ne.write(b)

Ask for the prediction...

In [ ]:
import json

object_detector.content_type = 'image/jpeg'
results = object_detector.predict(b)
detections = json.loads(results)
print (detections)



In [ ]:
def visualize_detection(img_file, dets, classes=[], thresh=0.90):
        """
        visualize detections in one image
        Parameters:
        ----------
        img : numpy.array
            image, in bgr format
        dets : numpy.array
            ssd detections, numpy.array([[id, score, x1, y1, x2, y2]...])
            each row is one object
        classes : tuple or list of str
            class names
        thresh : float
            score threshold
        """
        import random
        import matplotlib.pyplot as plt
        import matplotlib.image as mpimg

        img=mpimg.imread(img_file)
        plt.imshow(img)
        height = img.shape[0]
        width = img.shape[1]
        colors = dict()
        for det in dets:
            (klass, score, x0, y0, x1, y1) = det
            if score < thresh:
                continue
            cls_id = int(klass)
            if cls_id not in colors:
                colors[cls_id] = (random.random(), random.random(), random.random())
            xmin = int(x0 * width)
            ymin = int(y0 * height)
            xmax = int(x1 * width)
            ymax = int(y1 * height)
            rect = plt.Rectangle((xmin, ymin), xmax - xmin,
                                 ymax - ymin, fill=False,
                                 edgecolor=colors[cls_id],
                                 linewidth=3.5)
            plt.gca().add_patch(rect)
            class_name = str(cls_id)
            if classes and len(classes) > cls_id:
                class_name = classes[cls_id]
            plt.gca().text(xmin, ymin - 2,
                            '{:s} {:.3f}'.format(class_name, score),
                            bbox=dict(facecolor=colors[cls_id], alpha=0.5),
                                    fontsize=12, color='white')
        plt.show()

Finally visualize the prediction. How did you do? You can change the threshold and re-run this frame.

In [ ]:
object_categories = ['ship']
# Setting a threshold 0.20 will only plot detection results that have a confidence score greater than 0.20.
threshold = 0.50

# Visualize the detections.
visualize_detection(file_name, detections['prediction'], object_categories, threshold)